In [1]:
import csv
import re
from datetime import datetime
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from requests_html import HTMLSession
import nest_asyncio

In [2]:
frameworks = ['Frontend','Mobile','Backend','Software-Engineering/OOP','Data Science', 'Low Level',
              'Databases','Cloud Computing','AI / ML','Testing/Utility', 'Animation']
skill_dict = {} #Initialize our dictionary with frameworks as keys and empty lists as values
for i in frameworks:
    skill_dict[i] = []


## Generate a url with our input parameters

In [3]:
def get_url(position,location,date):
    template = 'https://www.indeed.com/jobs?q={}&l={}&fromage={}'
    url = template.format(position,location,date)
    return url

## Generate full description url from first url

In [ ]:
def get_details_url(url,advn,vjk):
    template = url + '&advn={}&vjk={}'
    details_url = template.format(advn,vjk)
    return details_url

In [5]:
url = get_url('software','california','7')

In [ ]:
def main (position, location):
    job_desc = [] #list containing all descriptions
    length = len(cards)
    
    while True:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'jobsearch-SerpJobCard')
        job_desc = [] #list containing all descriptions
        length = len(cards)
        
        for x in range(length):
            advn = cards[x].get('data-empn')
            vjk = cards[x].get('data-jk')
            details_url = get_details_url(url,advn,vjk) #append and generate new Url

            #Open and load page using webdriver so we can get parse elements that load after our initial request

            browser = webdriver.Chrome(r'C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe')  
            browser.get(details_url)
            htmlSource = browser.page_source
            txt = BeautifulSoup(htmlSource, 'html.parser')
            browser.close()
            job_desc.append(txt.find('div','jobsearch-jobDescriptionText'))  #Append the full description of the job to our list
            job_desc.append('#---------------------------------------------------------------------------------------------------#')
            print(details_url)
            job_desc.append('#---------------------------------------------------------------------------------------------------#')

        # print(job_desc) 
        
        try:
            url = 'https://www.indeed.com' + soup.find('a',{'aria-label': 'Next'}).get('href')
        except AttributeError:
            break
    


In [6]:
###### Getting all the jobs cards to find their IDs for full desc.
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
cards = soup.find_all('div', 'jobsearch-SerpJobCard') 

In [7]:
##### Loop thru all the jobs and gather descriptions

job_desc = [] #list containing all descriptions
length = len(cards)
for x in range(length):
    advn = cards[x].get('data-empn')
    vjk = cards[x].get('data-jk')
    details_url = get_details_url(url,advn,vjk) #append and generate new Url

    #Open and load page using webdriver so we can get parse elements that load after our initial request
    
    browser = webdriver.Chrome(r'C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe')  
    browser.get(details_url)
    htmlSource = browser.page_source
    txt = BeautifulSoup(htmlSource, 'html.parser')
    browser.close()
    job_desc.append(txt.find('div','jobsearch-jobDescriptionText'))  #Append the full description of the job to our list
    job_desc.append('#---------------------------------------------------------------------------------------------------#')
    print(details_url)
    job_desc.append('#---------------------------------------------------------------------------------------------------#')
        
# print(job_desc) 
    

https://www.indeed.com/jobs?q=software&l=california&fromage=7&advn=9257281766953058&vjk=37b7a6027ef55921
https://www.indeed.com/jobs?q=software&l=california&fromage=7&advn=3399501605239740&vjk=3a31b3bd15307f87
https://www.indeed.com/jobs?q=software&l=california&fromage=7&advn=None&vjk=b306a1ebf5c47a9e
https://www.indeed.com/jobs?q=software&l=california&fromage=7&advn=2556084698700334&vjk=7e4807b89c872ba9
https://www.indeed.com/jobs?q=software&l=california&fromage=7&advn=2556084698700334&vjk=f76dbac531b6b991
https://www.indeed.com/jobs?q=software&l=california&fromage=7&advn=5809567387809865&vjk=89b44a68eedc84f4
https://www.indeed.com/jobs?q=software&l=california&fromage=7&advn=None&vjk=efcf48300d1bad2d
https://www.indeed.com/jobs?q=software&l=california&fromage=7&advn=None&vjk=e772e9abb572d047
https://www.indeed.com/jobs?q=software&l=california&fromage=7&advn=8690912762161442&vjk=98e292d6494e9569
https://www.indeed.com/jobs?q=software&l=california&fromage=7&advn=None&vjk=7290dfd840b35c5

In [8]:
# import nltk
# from nltk.tokenize import sent_tokenize, word_tokenize
# from nltk.corpus import PlaintextCorpusReader
# string1 = ''.join(map(str,job_desc))
# cleanText = nltk.Text(nltk.word_tokenize(string1))
# # corp = PlaintextCorpusReader(job_desc)
# # cleanText = nltk.Text(corp.words())
# match = cleanText.concordance('Java')
# match2 = cleanText.concordance('ReactJS')
# string2 = ''.join(map(str,job_desc))
# print(match)
# print(match2)
# if 'at' in listOfStrings :

In [27]:
skill_lst_pattern = ['Java','C#','Python','ReactJS','VueJS','AngularJS','React','Vue','Angular','CSS','HTML',
              'nodeJS','JS','PHP','node.js','mongoDB','MongoDB','SQL','NoSQL','ARM','Assembly','Arduino',
              'Raspberry Pi','Pi','GraphQL','Aws','AWS','Cloud','Automation','Testing','Unit Testing',
              'Integration Testing','CMS','Maven','Kotlin','Headless','iOS','IOS','Android','Flutter','Linux','Server',
              'Data Processing','Data Cleaning','Data Mining','Data Science','Docker','PostgreSQL','Typescript','Restful',
              'API','Ruby','Ruby on Rails','ES6','ES5','Json','Android Studio','React Native','Azure','UI/UX','UI','UX',
              'Gatsby','Go','Golang','Dart','Javascript','JavaScript','Cuda','Unix','Gpu','GPU','Big Data','ASP',
              'Asp.net','Objective-C','Objective C','Animation','Graphics','Perl','Laravel','XML','Swift','Scala',
              'Rust','Jquery','JQuery','Bootstrap','Git','Github','MVC','AI','ML','Machine Learning','Artificial Intelligence'
              ]
cleantxt = ''.join(map(str,job_desc))
x = "s C++ C/C++ and .net and .Net and node.js and Java and UI/UX and C and C++ and C/C++s and node.js and . and c capital"
match1 = (re.findall(r"(?=("+'|'.join(skill_lst_pattern)+r"))", cleantxt)) # Matches our list of skills
match2 = (re.findall("\.Net|C\/C\+\+|\.net|node\.js|Node\.js|ASP\.Net |C\+\+|C\ ", cleantxt)) # Matches C/C++ and C and .net etc ...
skill_lst = match1+match2 # Final result of both our regex matches
print(skill_lst)


['Go', 'Java', 'HTML', 'ML', 'CSS', 'ReactJS', 'JS', 'AngularJS', 'JS', 'UI', 'GraphQL', 'Assembly', 'Java', 'ReactJS', 'JS', 'AngularJS', 'JS', 'UI', 'API', 'API', 'Testing', 'UI', 'UI', 'UI', 'JS', 'Angular', 'React', 'Vue', 'C#', 'Python', 'Git', 'Git', 'SQL', 'NoSQL', 'SQL', 'Linux', 'Unix', 'Azure', 'AWS', 'API', 'UI', 'Java', 'Python', 'Server', 'Testing', 'Python', 'Java', 'Java', 'SQL', 'Python', 'C#', 'SQL', 'Python', 'Cloud', 'Cloud', 'Go', 'Java', 'Git', 'Git', 'Git', 'Go', 'Go', 'Go', 'AWS', 'Azure', 'Docker', 'Docker', 'AWS', 'ML', 'AI', 'API', 'AI', 'API', 'HTML', 'ML', 'Java', 'CSS', 'Java', 'SQL', 'NoSQL', 'SQL', 'UI/UX', 'UX', 'ASP', 'ASP', 'ASP', 'UI', 'UX', 'ASP', 'C#', 'SQL', 'ASP', 'UI', 'UI', 'JS', 'Angular', 'React', 'Vue', 'C#', 'Python', 'Git', 'Git', 'SQL', 'NoSQL', 'SQL', 'Linux', 'Unix', 'Azure', 'AWS', 'API', 'Python', 'Python', 'C ', 'C++', '.Net', 'C ', 'ASP.Net ', 'C++', 'C++', 'C++', 'C ']
